In [3]:
# Change working directory
os.chdir('/Users/suongsuong/Documents/GitHub/Reactivity-based-metric-of-complexity/Reduction of ketone/')

import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem 

In [4]:
def generate_3D(data, folder_path):
    id_toremove = [] 
    for idx, row in data.iterrows():
        smiles = row['Reactant smiles map']
        reaction_id = row['Reaction ID']
        mol = Chem.MolFromSmiles(smiles) 
        mol = Chem.AddHs(mol)
        embed_success = AllChem.EmbedMolecule(mol,useRandomCoords=True, maxAttempts = 500)
        if embed_success == -1:
            print(f'Embedding failed for Reaction ID: {reaction_id}')
            id_toremove.append(reaction_id)
            continue
        try:
            AllChem.UFFOptimizeMolecule(mol)
            xyz_filename = folder_path + str(reaction_id) + '.xyz'
            Chem.MolToXYZFile(mol, xyz_filename)
        except ValueError as e:
            print(f'Optimization failed for Reaction ID: {reaction_id}, Error: {e}')
            id_toremove.append(reaction_id)
            continue 
    return id_toremove

In [5]:
atomMap_df = pd.read_excel('ReductionKetone_atomMapping.xlsx')
atomMap_df.head(5)

,Reaction,Reactant,Product,Reagent,Catalyst,Solvent (Reaction Details),Time (Reaction Details) [h],Temperature (Reaction Details) [C],Yield,Reaction ID,...,Largest Reactant MW,change in C-O single bond,change in C=O double bond,Procedure,Reactant MW,Reaction Map,Reactant smiles map,Product smiles map,C_idx,O_idx
0,COC(OC)C1=CC=CC=C1C(CC#N)C(=O)C1=CC=CC=C1>>COC...,(±)-3-[2-(dimethoxymethyl)phenyl]-4-oxo-4-phen...,"(±)-(3RS,4RS)-3-[2-(dimethoxymethyl)phenyl]-4-...",sodium tetrahydroborate,NaN,methanol,16.0,0,82 percent,38244374,...,309.365,1,-1,"Under N2, a solution of ketone 7 (1.41 g, 4.56...",309.365,[CH3:1][O:2][CH:3]([O:4][CH3:5])[c:6]1[cH:7][c...,[CH3:1][O:2][CH:3]([O:4][CH3:5])[c:6]1[cH:7][c...,[CH3:1][O:2][CH:3]([O:4][CH3:5])[c:6]1[cH:7][c...,16,17
1,BrC1=CC=C(C=C1)C(=O)CN1C=C(N=N1)C1=CC=CC=C1>>O...,"1-(4-bromophenyl)-2-(4-phenyl-1H-1,2,3-triazol...",C16H14BrN3O,sodium tetrahydroborate,NaN,methanol,1.5,0,79 percent,46506389,...,342.196,1,-1,The 2-azido-1-phenylethanols 1a–4a were obtain...,342.196,[Br:19][c:18]1[cH:17][cH:16][c:15]([C:2](=[O:1...,[O:1]=[C:2]([CH2:3][n:4]1[cH:5][c:6](-[c:9]2[c...,[OH:1][CH:2]([CH2:3][n:4]1[cH:5][c:6](-[c:9]2[...,2,1
2,O=C(CN1C=C(N=N1)C1=CC=CC=C1)C1=CC=CC=C1>>OC(CN...,"1-phenyl-2-(4-phenyl-[1,2,3]triazole-1-yl)etha...","1-phenyl-2-(4-phenyl-1H-1,2,3-triazol-1-yl)eth...",sodium tetrahydroborate,NaN,methanol,1.5,0,91 percent,34623882,...,263.300,1,-1,The 2-azido-1-phenylethanols 1a–4a were obtain...,263.300,[O:1]=[C:2]([CH2:3][n:4]1[cH:5][c:6](-[c:9]2[c...,[O:1]=[C:2]([CH2:3][n:4]1[cH:5][c:6](-[c:9]2[c...,[OH:1][CH:2]([CH2:3][n:4]1[cH:5][c:6](-[c:9]2[...,2,1
3,ClC1=CC=C(C=C1)C(=O)CN1C=C(N=N1)C1=CC=CC=C1>>O...,"1-(4-chlorophenyl)-2-(4-phenyl-1H-1,2,3-triazo...",C16H14ClN3O,sodium tetrahydroborate,NaN,methanol,1.5,0,80 percent,34623878,...,297.745,1,-1,The 2-azido-1-phenylethanols 1a–4a were obtain...,297.745,[Cl:19][c:18]1[cH:17][cH:16][c:15]([C:2](=[O:1...,[O:1]=[C:2]([CH2:3][n:4]1[cH:5][c:6](-[c:9]2[c...,[OH:1][CH:2]([CH2:3][n:4]1[cH:5][c:6](-[c:9]2[...,2,1
4,COC1=CC=C(C=C1)C(=O)CN1C=C(N=N1)C1=CC=CC=C1>>C...,"1-(4-methoxyphenyl)-2-(4-phenyl-1H-1,2,3-triaz...","1-(4-methoxyphenyl)-2-(4-phenyl-1H-1,2,3-triaz...",sodium tetrahydroborate,NaN,methanol,1.5,0,92 percent,34623890,...,293.326,1,-1,The 2-azido-1-phenylethanols 1a–4a were obtain...,293.326,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6]([C:9](=[O:...,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6]([C:9](=[O:...,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6]([CH:9]([OH...,7,8


------ 
#### generate 3D structure

In [6]:
# atomMap_df = atomMap_df.head(100)
id_remove = generate_3D(atomMap_df, '3D_structures/')
df_3D = atomMap_df[~atomMap_df['Reaction ID'].isin(id_remove)]  # remove unsuccessful 3D

Embedding failed for Reaction ID: 32575260
Embedding failed for Reaction ID: 48314851
Embedding failed for Reaction ID: 9348888
Embedding failed for Reaction ID: 27941778
Embedding failed for Reaction ID: 50412793
Embedding failed for Reaction ID: 39813135
Embedding failed for Reaction ID: 29204447
Embedding failed for Reaction ID: 29659677
Embedding failed for Reaction ID: 3695571
Embedding failed for Reaction ID: 27941777
Embedding failed for Reaction ID: 27941779
Embedding failed for Reaction ID: 50276052
Embedding failed for Reaction ID: 42625109
Embedding failed for Reaction ID: 32414566
Embedding failed for Reaction ID: 32414574
Embedding failed for Reaction ID: 32414633
Embedding failed for Reaction ID: 32815563
Embedding failed for Reaction ID: 32815590
Embedding failed for Reaction ID: 33075553
Embedding failed for Reaction ID: 33075662
Embedding failed for Reaction ID: 35947119
Embedding failed for Reaction ID: 35947155
Embedding failed for Reaction ID: 35947187
Embedding fai

In [7]:
print('There are',len(id_remove),'rxn not having generated 3D structures')

There are 56 rxn not having generated 3D structures


------ 
### make sure the right index was pulled out

#### read xyz file to see the index 
first index represent the first atom in the xyz file

In [11]:
def read_xyz_file_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Skip the first two lines (header lines)
    atom_lines = lines[2:]
    
    # Initialize lists to store data
    count = 0
    atom_count = []
    atoms = []
    x_coords = []
    y_coords = []
    z_coords = []

    for line in atom_lines:
        count +=1
        parts = line.split()
        atom_type = parts[0]
        x = float(parts[1])
        y = float(parts[2])
        z = float(parts[3])
        
        atom_count.append(count)
        atoms.append(atom_type)
        x_coords.append(x)
        y_coords.append(y)
        z_coords.append(z)
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Atom Num': atom_count,
        'Atom': atoms,
        'X': x_coords,
        'Y': y_coords,
        'Z': z_coords
    })
    
    return df
read_xyz_file_to_dataframe('3D_structures/40358815.xyz'  )

,Atom Num,Atom,X,Y,Z
0,1,C,-0.603131,0.827603,2.366694
1,2,C,0.001600,-0.370823,3.134838
2,3,C,1.267548,-0.879639,2.466281
3,4,C,0.962574,-1.349932,1.079583
4,5,C,1.770825,-2.370506,0.530248
5,6,C,1.497847,-2.812161,-0.756442
6,7,C,0.472731,-2.270513,-1.492765
7,8,C,-0.335408,-1.253929,-0.999188
8,9,C,-0.094378,-0.759243,0.313531
9,10,C,-0.949596,0.414697,0.859136


In [12]:
df_3D[df_3D['Reaction ID']==40358815]

,Reaction,Reactant,Product,Reagent,Catalyst,Solvent (Reaction Details),Time (Reaction Details) [h],Temperature (Reaction Details) [C],Yield,Reaction ID,...,Largest Reactant MW,change in C-O single bond,change in C=O double bond,Procedure,Reactant MW,Reaction Map,Reactant smiles map,Product smiles map,C_idx,O_idx
5,[H][C@]12CCC3=CC4=C(C=C3[C@]1(C)CCC(=O)[C@@]2(...,C24H25NO3,C24H27NO3,sodium tetrahydroborate,NaN,methanol,3.0,0,72 percent,40358815,...,375.468,1,-1,"Alcohol S6. Carbazole 18 (100 mg, 0.27 mmol, 1...",375.468,[C@H:1]12[CH2:2][CH2:3][c:4]3[cH:5][c:6]4[c:7]...,[C@@H:1]12[CH2:2][CH2:3][c:4]3[cH:5][c:6]4[c:7...,[C@@H:1]12[CH2:2][CH2:3][c:4]3[cH:5][c:6]4[c:7...,14,15


In [ ]:
# # USE THIS CODE, combinded with view_reactionScheme() with mapping, to check if the right index was pull out
# df = get_sterimol[get_sterimol['Reaction ID'] == 52106508]
# for idx, row in df.iterrows(): 
#     smiles_map = row['Reactant smiles map']
#     reactant_mol = Chem.MolFromSmiles(smiles_map)

#     C_idx = row['C_idx'] - 1 # - 1 because the idx in the df is the order of atom counting from 1
#     O_idx = row['O_idx'] - 1
#     atom_C = reactant_mol.GetAtomWithIdx(C_idx)
#     atom_O = reactant_mol.GetAtomWithIdx(O_idx)
#     print('this supposed to be C:', atom_C.GetSymbol(), atom_C.GetAtomMapNum())
#     print('this supposed to be O:', atom_O.GetSymbol(), atom_O.GetAtomMapNum())

----

In [13]:
df_3D.to_excel('ReductionKetone_have3D.xlsx',index=False)